In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import plotly.graph_objects as go
import plotly.express as px
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb
import optuna
import os
import shap
import pickle


from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sksurv.metrics import concordance_index_censored

In [ ]:
# Test GPU par XGBoost
try:
    # On crée une micro-matrice de test
    data = xgb.DMatrix([[1, 2], [3, 4]], label=[1, 0])

    params = {'tree_method': 'gpu_hist', 'device': 'cuda'}
    xgb.train(params, data, num_boost_round=1)
    print("✅ Succès ! La RTX 4060 est reconnue et configurée.")
except Exception as e:
    print(f"❌ Échec du GPU : {e}")
    print("Le modèle tournera sur CPU par défaut.")

In [ ]:
df = pd.read_parquet('dataset_full.parquet')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Structure du dataset : {df.shape[0]} lignes et {df.shape[1]} colonnes")
display(df.head())

In [ ]:
# --- 1. GÉOGRAPHIE ---
df['dep'] = df["Code du département de l'établissement"].astype(str).str.zfill(2)

# --- 2. TRAITEMENT DE BASE ---
df['Catégorie juridique de l\'unité légale'] = df['Catégorie juridique de l\'unité légale'].astype(str)
df['Tranche_effectif_num'] = df['Tranche_effectif_num'].fillna(0).astype(float)
df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0)

# --- 3. NOUVELLE FEATURE PERTINENTE ---
# On calcule la taille moyenne par secteur APE
secteur_moyennes = df.groupby('libelle_section_ape')['Tranche_effectif_num'].transform('mean')
# Est-ce que la boîte est plus costaude que la moyenne de son secteur ?
df['taille_relative_secteur'] = df['Tranche_effectif_num'] - secteur_moyennes

# --- 4. ENCODAGE ---
df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', "Catégorie juridique de l'unité légale"], 
    prefix=['APE', 'CJ'],
    drop_first=True
)

# --- 5. NETTOYAGE RIGOUREUX ---
# On retire tout ce qui n'est pas une feature d'apprentissage
cols_to_drop = [
    'SIREN', 'Code postal de l\'établissement', 'Code commune de l\'établissement',
    'Dénomination de l\'unité légale', 'Activité principale de l\'unité légale',
    'Date_fermeture_finale', 'latitude', 'longitude', 'code_ape',
    'Code du département de l\'établissement', 'Code de la région de l\'établissement',
    'Economie sociale et solidaire unité légale', 'dep',
    'densite_salariale' # AU CAS OÙ ELLE TRAÎNE ENCORE
]

# X contient les features, y_time la survie, y_event l'état (ouvert/fermé)
X = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns] + ['fermeture', 'age_estime'])
y_time = df_final['age_estime']
y_event = df_final['fermeture'].astype(bool)

print(f"✅ Features (X) prêtes sans leakage : {X.shape[1]} colonnes.")

In [ ]:
# 1. NETTOYAGE GÉOGRAPHIQUE
df['dep'] = df["Code du département de l'établissement"].astype(str).str.zfill(2)

# 2. TRAITEMENT DES TYPES
df["Catégorie juridique de l'unité légale"] = df["Catégorie juridique de l'unité légale"].astype(str)
df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0)

# 3. ENCODAGE (One-Hot)
df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', "Catégorie juridique de l'unité légale"], 
    prefix=['APE', 'CJ'],
    drop_first=True
)

# 4. NETTOYAGE DES CATÉGORIES RARES (< 0.1%)
binary_cols = [c for c in df_final.columns if c.startswith('APE_') or c.startswith('CJ_')]
frequencies = df_final[binary_cols].mean()
rare_cols = frequencies[frequencies < 0.001].index.tolist()
df_final = df_final.drop(columns=rare_cols)
print(f"✂️ {len(rare_cols)} colonnes rares supprimées.")

# 5. SÉLECTION FINALE ET PRÉPARATION DES CIBLES

cols_a_exclure = [
    'SIREN', 'Code postal de l\'établissement', 'Code commune de l\'établissement',
    'Dénomination de l\'unité légale', 'Activité principale de l\'unité légale',
    'Date_fermeture_finale', 'latitude', 'longitude', 'code_ape',
    'Code de la région de l\'établissement', 'Economie sociale et solidaire unité légale',
    'age_estime', 'fermeture', 
    'densite_salariale',        # Contient l'âge (Leakage)
    'Tranche_effectif_num',    # Prédit trop bien la fermeture imminente (Leakage)
    'taille_relative_secteur'  # Dérivé de l'effectif (Leakage)
]

# On crée X. On garde 'dep' (il n'est pas dans cols_a_exclure ici) pour le split.
X = df_final.drop(columns=[c for c in cols_a_exclure if c in df_final.columns])
y_time = df_final['age_estime']
y_event = df_final['fermeture'].astype(bool)

# Filtrage des âges incohérents
mask = y_time > 0
X = X[mask]
y_time = y_time[mask]
y_event = y_event[mask]

print(f"✅ Dataset ENFIN honnête : {X.shape[0]} lignes, {X.shape[1]} colonnes (incluant 'dep').")

In [ ]:
# 1. SPLIT INITIAL (On sépare le Test final)
X_temp, X_test, y_time_temp, y_test_time, y_event_temp, y_test_event = train_test_split(
    X, y_time, y_event, test_size=0.15, random_state=42
)

# 2. DEUXIÈME SPLIT (Train vs Validation)
X_train, X_val, y_train_time, y_val_time, y_train_event, y_val_event = train_test_split(
    X_temp, y_time_temp, y_event_temp, test_size=0.176, random_state=42
)

# 3. CALCUL DU RISQUE DÉPARTEMENTAL
dep_risk_map = pd.concat([X_train, y_train_event], axis=1).groupby("dep")["fermeture"].mean()

X_train['risque_departemental'] = X_train['dep'].map(dep_risk_map)
X_val['risque_departemental'] = X_val['dep'].map(dep_risk_map)
X_test['risque_departemental'] = X_test['dep'].map(dep_risk_map)

global_mean = y_train_event.mean()
X_train['risque_departemental'] = X_train['risque_departemental'].fillna(global_mean)
X_val['risque_departemental'] = X_val['risque_departemental'].fillna(global_mean)
X_test['risque_departemental'] = X_test['risque_departemental'].fillna(global_mean)

# 4. NETTOYAGE FINAL (Modifié pour supprimer TOUT le texte génant)

cols_a_supprimer = ['dep', "Code du département de l'établissement", "libelle_section_ape"]

for df_set in [X_train, X_val, X_test]:
    # On ne supprime que si la colonne existe
    existantes = [c for c in cols_a_supprimer if c in df_set.columns]
    df_set.drop(columns=existantes, inplace=True)

print(f"📊 Train : {len(X_train)} lignes | Features: {X_train.shape[1]}")
print(f"🧪 Val   : {len(X_val)} lignes")
print(f"🔒 Test  : {len(X_test)} lignes")

In [ ]:
# On prépare les fonctions pour les bornes AFT UNIQUEMENT
def create_aft_inputs_clean(y_time, y_event, X):
    # On s'assure de ne pas envoyer la colonne 'dep' dans le modèle
    X_clean = X.drop(columns=['dep']) if 'dep' in X.columns else X
    
    # Bornes de survie
    y_lower = y_time.values
    y_upper = np.where(y_event == 1, y_time.values, np.inf)
    
    # Création de la matrice
    dmat = xgb.DMatrix(X_clean)
    
    # On injecte les bornes MAIS PAS le base_margin (pour éviter le leakage)
    dmat.set_info(
        label_lower_bound=y_lower,
        label_upper_bound=y_upper
    )
    return dmat

# 1. On prépare les matrices SANS margin
dtrain = create_aft_inputs_clean(y_train_time, y_train_event, X_train)
dval   = create_aft_inputs_clean(y_val_time, y_val_event, X_val)

# 2. Pour le test (pareil, pas de margin)
X_test_clean = X_test.drop(columns=['dep']) if 'dep' in X_test.columns else X_test
dtest = xgb.DMatrix(X_test_clean)

print("✅ Ingrédients 3.0 prêts : Le modèle va maintenant apprendre à prédire la survie sans connaître l'âge à l'avance.")

In [ ]:
# # 1. CONFIGURATION CONNEXION HUGGING FACE
# load_dotenv()

# mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
# mlflow_user = os.getenv("MLFLOW_TRACKING_USERNAME")
# mlflow_pass = os.getenv("MLFLOW_TRACKING_PASSWORD")

# mlflow.set_tracking_uri(mlflow_uri)
# os.environ['MLFLOW_TRACKING_USERNAME'] = mlflow_user
# os.environ['MLFLOW_TRACKING_PASSWORD'] = mlflow_pass

# mlflow.set_experiment("Survival_AFT_With_Age_Margin")

# def objective(trial):
#     params = {
#         'objective': 'survival:aft',
#         'eval_metric': 'aft-nloglik',
#         'tree_method': 'hist',
#         'device': 'cuda', # Ta 4060 va travailler dur ici
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'aft_loss_distribution': trial.suggest_categorical('aft_loss_distribution', ['normal', 'logistic']),
#         'aft_loss_distribution_scale': trial.suggest_float('aft_loss_distribution_scale', 0.5, 2.0),
#     }

#     with mlflow.start_run(nested=True):
#         # Entraînement
#         bst = xgb.train(
#             params, 
#             dtrain, 
#             num_boost_round=2000,
#             evals=[(dval, 'val')],
#             early_stopping_rounds=50,
#             verbose_eval=False
#         )
#         preds = bst.predict(dval)
#         c_index = concordance_index_censored(
#             y_val_event.astype(bool), 
#             y_val_time, 
#             -preds
#         )[0]
        
#         # Logs MLflow
#         mlflow.log_params(params)
#         mlflow.log_metric("c_index_val", c_index)
#         mlflow.log_metric("best_iteration", bst.best_iteration)
        
#         return c_index

# # 2. LANCEMENT DE L'OPTIMISATION
# with mlflow.start_run(run_name="AFT_Search_Final_Features"):
#     study = optuna.create_study(direction='maximize')
#     study.optimize(objective, n_trials=50)

# print(f"🏆 Meilleur C-Index : {study.best_value}")
# print(f"📊 Meilleurs paramètres : {study.best_params}")

In [ ]:
# 1. On entraîne le modèle final avec les paramètres du Trial 21
best_params = study.best_params
best_params.update({
    'objective': 'survival:aft',
    'tree_method': 'hist',
    'device': 'cuda'
})

# Entraînement un peu plus long pour stabiliser les scores d'importance
final_model = xgb.train(best_params, dtrain, num_boost_round=1000)

# 2. Extraction du GAIN (Contribution à la précision)
importance_gain = final_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'Feature': importance_gain.keys(),
    'Gain': importance_gain.values()
}).sort_values(by='Gain', ascending=False)

# 3. Visualisation du Top 15
plt.figure(figsize=(12, 8))
sns.barplot(data=importance_df.head(15), x='Gain', y='Feature', palette='flare')
plt.title("🏆 Top 15 des Variables les plus Prédictives (Modèle Honnête)")
plt.xlabel("Importance (Gain moyen par split)")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Extraction des scores
importance_dict = final_model.get_score(importance_type='gain')

# Création d'un DataFrame pour un affichage propre
df_importance = pd.DataFrame({
    'Feature': list(importance_dict.keys()),
    'Gain_Score': list(importance_dict.values())
})

# Tri par importance décroissante
df_importance = df_importance.sort_values(by='Gain_Score', ascending=False)

# Affichage du Top 20
print("🏆 TOP 20 DES FEATURES (PAR GAIN) :")
print(df_importance.head(20).to_string(index=False))

In [ ]:


# 1. Calcul des SHAP values (explication de la direction)
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_val.drop(columns=['dep']) if 'dep' in X_val.columns else X_val)

# 2. Visualisation du Summary Plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_val.drop(columns=['dep']) if 'dep' in X_val.columns else X_val, plot_type="bar")

In [ ]:
import pandas as pd
import numpy as np

def generer_palmares(model, X_template):
    palmares = []
    
    # On récupère toutes les colonnes qui commencent par APE_
    cols_ape = [c for c in X_template.columns if c.startswith('APE_')]
    
    print(f"Analyse de {len(cols_ape)} secteurs d'activité...")

    for ape in cols_ape:
        # 1. On crée un profil moyen (toutes les colonnes à 0)
        profil = pd.DataFrame(0, index=[0], columns=X_template.columns)
        
        # 2. On active uniquement le secteur APE en cours
        profil[ape] = 1
        
        # 3. On fixe les autres variables sur une valeur neutre (médiane)
        if 'dep_risk_map' in profil.columns:
            profil['dep_risk_map'] = X_template['dep_risk_map'].median()
        
        # 4. Prédiction log-temps
        dmat = xgb.DMatrix(profil)
        log_life = model.predict(dmat)[0]
        
        # 5. Conversion en années
        annees = np.exp(log_life)
        
        palmares.append({
            'Secteur': ape.replace('APE_', ''),
            'Esperance_Vie_Mediane': round(annees, 2)
        })

    # Création du DataFrame final
    df_palmares = pd.DataFrame(palmares).sort_values(by='Esperance_Vie_Mediane', ascending=False)
    return df_palmares

# --- EXÉCUTION ---
# On utilise X_train pour avoir le template des colonnes
df_resultat = generer_palmares(final_model, X_train.drop(columns=['dep']) if 'dep' in X_train.columns else X_train)

print("\n💎 LES 5 SECTEURS LES PLUS ROBUSTES :")
print(df_resultat.head(5).to_string(index=False))

print("\n⚠️ LES 5 SECTEURS LES PLUS FRAGILES :")
print(df_resultat.tail(5).to_string(index=False))

In [ ]:
def generer_palmares_robuste(model, X_template):
    palmares = []
    cols_ape = [c for c in X_template.columns if c.startswith('APE_')]
    
    # On définit un profil "moyen" pour tout le monde
    base_profil = pd.DataFrame(0, index=[0], columns=X_template.columns)
    if 'dep_risk_map' in base_profil.columns:
        base_profil['dep_risk_map'] = X_template['dep_risk_map'].median()

    for ape in cols_ape:
        profil = base_profil.copy()
        profil[ape] = 1
        
        # On récupère le score brut (log-life)
        dmat = xgb.DMatrix(profil)
        log_life = model.predict(dmat)[0]
        
        palmares.append({
            'Secteur': ape.replace('APE_', ''),
            'Score_Survie': log_life  # On garde le score brut pour le tri
        })

    df = pd.DataFrame(palmares).sort_values(by='Score_Survie', ascending=False)
    
    # On borne l'exponentielle pour l'affichage (max 100 ans pour rester réaliste)
    df['Vie_Estimee_Ans'] = df['Score_Survie'].apply(lambda x: np.exp(min(x, 4.6))) # 4.6 ~ log(100)
    
    return df

# Relance le palmarès
df_resultat = generer_palmares_robuste(final_model, X_train.drop(columns=['dep']) if 'dep' in X_train.columns else X_train)

print("\n🏆 CLASSEMENT DES SECTEURS (Du plus solide au plus fragile) :")
print(df_resultat[['Secteur', 'Score_Survie']].head(10)) # Top 10
print("...")
print(df_resultat[['Secteur', 'Score_Survie']].tail(10)) # Bottom 10

---

##### Nouvelle tentative de train avec prise en comtpe de l'age pour API

In [ ]:
# # 1. CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# load_dotenv()

# mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
# mlflow_user = os.getenv("MLFLOW_TRACKING_USERNAME")
# mlflow_pass = os.getenv("MLFLOW_TRACKING_PASSWORD")

# # Configuration de la connexion distante
# mlflow.set_tracking_uri(mlflow_uri)
# os.environ['MLFLOW_TRACKING_USERNAME'] = mlflow_user
# os.environ['MLFLOW_TRACKING_PASSWORD'] = mlflow_pass

# # 2. PRÉPARATION DES DONNÉES (X_api avec l'âge)
# # On s'assure de prendre l'âge estimé et les colonnes déjà encodées dans X
# X_api = df_final[[c for c in df_final.columns if c in X.columns or c == 'age_estime']].copy()

# # Nettoyage automatique des colonnes non-numériques (Sécurité pour XGBoost)
# for col in X_api.columns:
#     if X_api[col].dtype == 'object':
#         try:
#             X_api[col] = pd.to_numeric(X_api[col])
#         except ValueError:
#             print(f"Suppression de la colonne incompatible : {col}")
#             X_api.drop(columns=[col], inplace=True)

# # Cible : 1 si fermé, 0 si ouvert
# y_api = (df_final['fermeture'] == 1).astype(int) 

# # Split Entraînement / Validation
# X_train_api, X_val_api, y_train_api, y_val_api = train_test_split(
#     X_api, y_api, test_size=0.2, random_state=42, stratify=y_api
# )

# # 3. ENTRAÎNEMENT ET TRACKING MLFLOW
# mlflow.set_experiment("Business_Risk_Classifier_API")

# # On active l'autolog pour capturer les courbes d'apprentissage automatiquement
# mlflow.xgboost.autolog()

# with mlflow.start_run(run_name="XGB_Classifier_With_Age"):
#     # Définition des paramètres
#     params = {
#         "n_estimators": 1000,
#         "learning_rate": 0.05,
#         "max_depth": 6,
#         "early_stopping_rounds": 50,
#         "tree_method": 'hist',
#         "device": 'cuda', # Utilise ton GPU
#         "eval_metric": 'auc'
#     }
    
#     # Log des paramètres manuels (en plus de l'autolog)
#     mlflow.log_params(params)

#     # Création du modèle
#     api_model = XGBClassifier(**params)

#     # Fit
#     api_model.fit(
#         X_train_api, y_train_api,
#         eval_set=[(X_val_api, y_val_api)],
#         verbose=100
#     )

#     # Calcul des prédictions pour le score final
#     preds_proba = api_model.predict_proba(X_val_api)[:, 1]
#     auc_score = roc_auc_score(y_val_api, preds_proba)
    
#     # Log de la métrique finale
#     mlflow.log_metric("final_auc_val", auc_score)
    
#     # Sauvegarde du modèle en tant qu'artefact MLflow
#     mlflow.xgboost.log_model(api_model, artifact_path="risk_classifier_model")
    
#     print(f"\n✅ Entraînement terminé !")
#     print(f"🚀 Précision (AUC) : {auc_score:.4f}")
#     print(f"📍 Consultable sur : {mlflow_uri}")

---

#### Test de predictions

In [ ]:
def predict_business_risk(age, code_ape, dep_risk_score=None, is_ess=0):
    """
    Fonction de prédiction alignée sur les colonnes réelles du modèle.
    """
    # 1. On crée le DataFrame avec l'exacte liste des colonnes d'entraînement
    input_df = pd.DataFrame(0, index=[0], columns=api_model.feature_names_in_)
    
    # 2. On remplit les variables de base si elles existent dans le modèle
    if 'age_estime' in input_df.columns:
        input_df['age_estime'] = age
    if 'is_ess' in input_df.columns:
        input_df['is_ess'] = is_ess
    if 'dep_risk_map' in input_df.columns and dep_risk_score is not None:
        input_df['dep_risk_map'] = dep_risk_score
        
    # 3. On active le code APE
    column_name = f"APE_{code_ape}"
    if column_name in input_df.columns:
        input_df[column_name] = 1
    else:
        print(f"⚠️ Attention : Le secteur '{column_name}' n'est pas reconnu par le modèle.")
    
    # 4. Prédiction
    proba = api_model.predict_proba(input_df)[0][1]
    
    return {
        "score_risque": round(float(proba) * 100, 2),
        "interpretation": "Risque Élevé" if proba > 0.5 else "Risque Modéré"
    }

# --- TEST ---
# On enlève taille_relative_secteur du test car le modèle ne l'a pas vu
test_pme = predict_business_risk(age=2, code_ape="Restauration", dep_risk_score=0.15)
print(f"✅ Résultat pour le test : {test_pme}")

In [ ]:
profils = [
    {"nom": "Restau Jeune (2 ans)", "age": 2, "ape": "Restauration"},
    {"nom": "Restau Ancien (15 ans)", "age": 15, "ape": "Restauration"},
    {"nom": "Électricité (5 ans)", "age": 5, "ape": "Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné"}
]

for p in profils:
    res = predict_business_risk(age=p['age'], code_ape=p['ape'])
    print(f"PROFIL: {p['nom']} --> Risque: {res['score_risque']}%")

In [ ]:
# 1. On prépare les données (mêmes colonnes que pour l'entraînement)

X_scoring = X_api[api_model.feature_names_in_]

# 2. On lance la prédiction sur l'ensemble du dataset
# predict_proba renvoie [probabilité de 0, probabilité de 1]
# On prend [:, 1] pour avoir la probabilité de FERMETURE
print("🚀 Calcul des scores de risque en cours...")
df_final['probabilite_fermeture'] = api_model.predict_proba(X_scoring)[:, 1]

# 3. Conversion en pourcentage pour plus de lisibilité
df_final['score_risque_pct'] = (df_final['probabilite_fermeture'] * 100).round(2)

# 4. Affichage des entreprises les plus à risque qui sont encore OUVERTES
print("\n🔥 Top 10 des entreprises encore ouvertes les plus à risque :")
entreprises_a_risque = df_final[df_final['fermeture'] == 0].sort_values(by='score_risque_pct', ascending=False)

print(entreprises_a_risque[["Dénomination de l'unité légale", 'age_estime', 'score_risque_pct']].head(10))

##### Test de predictions

In [ ]:
# On crée des copies pour simuler le futur
X_now = df_final[api_model.feature_names_in_].copy()
X_plus_1 = X_now.copy()
X_plus_2 = X_now.copy()
X_plus_3 = X_now.copy()

# On simule le vieillissement
X_plus_1['age_estime'] += 1
X_plus_2['age_estime'] += 2
X_plus_3['age_estime'] += 3

print("⏳ Calcul des prédictions temporelles...")

# On calcule les probabilités de FERMETURE pour chaque horizon
# Rappel : predict_proba()[:, 1] donne le risque de fermeture
df_final['risque_actuel'] = api_model.predict_proba(X_now)[:, 1]
df_final['risque_1_an'] = api_model.predict_proba(X_plus_1)[:, 1]
df_final['risque_2_ans'] = api_model.predict_proba(X_plus_2)[:, 1]
df_final['risque_3_ans'] = api_model.predict_proba(X_plus_3)[:, 1]

# Pour avoir la probabilité de SURVIE (ce qui est souvent plus parlant)
# Probabilité de survie = 1 - Risque de fermeture
df_final['survie_1_an_pct'] = ((1 - df_final['risque_1_an']) * 100).round(2)
df_final['survie_2_ans_pct'] = ((1 - df_final['risque_2_ans']) * 100).round(2)
df_final['survie_3_ans_pct'] = ((1 - df_final['risque_3_ans']) * 100).round(2)

# Affichage du résultat
cols_view = ["Dénomination de l'unité légale", 'age_estime', 'survie_1_an_pct', 'survie_2_ans_pct', 'survie_3_ans_pct']
print("\n📅 Prévisions de survie à 1, 2 et 3 ans :")
print(df_final[cols_view].head(10))

In [ ]:
df_final[cols_view].head(100)

#### Modification pour afficher un taux plus cohérent

In [ ]:
# 1. Calcul du risque de base (aujourd'hui) en probabilité (0 à 1)
proba_now = api_model.predict_proba(X_now)[:, 1]

# 2. On remplit les colonnes selon la structure attendue
df_final['Indice_Risque'] = proba_now.round(6)

# On applique la croissance du risque (l'inverse de ta décroissance de survie)
# Si survie = 95%, alors Risque = 1 - (Survie_Initiale * 0.95)
df_final['Prob_1an'] = (1 - ((1 - proba_now) * 0.95)).round(2)
df_final['Prob_2ans'] = (1 - ((1 - proba_now) * 0.95 * 0.92)).round(2)
df_final['Prob_3ans'] = (1 - ((1 - proba_now) * 0.95 * 0.92 * 0.88)).round(2)

# 3. Sécurité pour ne pas avoir de probabilités négatives
for col in ['Prob_1an', 'Prob_2ans', 'Prob_3ans']:
    df_final[col] = df_final[col].clip(lower=0)

# 4. Attribution du Statut_Expert (basé sur le risque à 3 ans)
def attribuer_statut(p):
    if p < 0.10: return '🟢 SAIN'
    elif p < 0.25: return '🟡 OBSERVATION'
    elif p < 0.50: return '🟠 VIGILANCE'
    else: return '🔴 CRITIQUE'

df_final['Statut_Expert'] = df_final['Prob_3ans'].apply(attribuer_statut)

# 5. Sélection et renommage des colonnes pour correspondre au tableau HTML
# Note : Assure-toi que les colonnes 'SIREN' et 'Dénomination' existent dans df_final
colonnes_finales = ['SIREN', "Dénomination de l'unité légale", 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans', 'Statut_Expert']
df_tableau = df_final[colonnes_finales].rename(columns={"Dénomination de l'unité légale": "Dénomination"})

print(df_tableau.head(10))

##### Sauvegarde du modèle

In [ ]:
# # --- SAUVEGARDE DU MODÈLE VERSION V3 ---

# # Option 1 : Format JSON (Compatibilité XGBoost pure)
# final_model.save_model("xgboost_v3.json")

# # Option 2 : Format Pickle (Format Python standard)
# with open("xgboost_v3.pkl", "wb") as f:
#     pickle.dump(final_model, f)

# print("✅ Modèle xgboost_v3 sauvegardé avec succès aux formats .json et .pkl !")